In [24]:
import pandas as pd
import json
import gzip
import ast

In [2]:
data = []
with gzip.open('steam_games.json.gz','rb') as f:
    for line in f:
        data.append(json.loads(line))
f.close()
games=pd.DataFrame(data)

In [4]:
games.shape

(120445, 13)

**Elimino del df las filas completamente nulas**

In [3]:
games.isnull().all(axis=1).value_counts() #cuento filas vacías

True     88310
False    32135
Name: count, dtype: int64

In [4]:
games_clean = games.dropna(axis=0, how='all')
games_clean.shape

(32135, 13)

**Paso a formato fecha la columna release_date**

In [6]:
games_clean.release_date= pd.to_datetime(games_clean.release_date, format= 'mixed', errors='coerce')
games_clean.release_date.iloc[0]

C:\Users\torin\AppData\Local\Temp\ipykernel_7804\750404850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_clean.release_date= pd.to_datetime(games_clean.release_date, format= 'mixed', errors='coerce')


Timestamp('2018-01-04 00:00:00')

In [65]:
games_clean.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro


In [66]:
#limpio algunas columnas que no me sirven
games_clean.drop(columns = ['publisher', 'url', 'reviews_url', 'specs', 'price', 'early_access', 'developer'], inplace=True)

C:\Users\torin\AppData\Local\Temp\ipykernel_7804\3280226977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_clean.drop(columns = ['publisher', 'url', 'reviews_url', 'specs', 'price', 'early_access', 'developer'], inplace=True)


In [68]:
#exporto el df final como un csv.
games_clean.to_csv('games_final.csv', index=False) 

In [7]:
data = []
with gzip.open('user_reviews.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))
f.close()
reviews = pd.DataFrame(data)

In [18]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


**Este código desanida la lista de reviews para cada usuario. La primer línea genera una fila por review de usuario dentro del dataframe. La segunda modifica el index para después poder concatenar. La tercer línea toma el diccionario dentro de cada review y lo normaliza, generando una columna para cada categoría dentro del review. La última une el dataframe que tiene una fila por review (con los datos del usuario) con el dataframe que tiene las columnas de las reviews**

In [8]:
final_reviews = reviews.explode('reviews') # armo una fila por cada elemento dentro de la lista reviews
final_reviews.index = range(len(final_reviews)) #reseteo el index
reviews_desanidados = pd.json_normalize(final_reviews.reviews) # armo un DF, con cada clave del diccionario dentro de reviews como una columna
reviews_clean = pd.concat([final_reviews.drop(columns= ['reviews']), reviews_desanidados], axis=1) #concateno el df extendido (con una fila por review de usuario) con el df con una columna por key de review

In [9]:
reviews_clean

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


**Este código limpia los datos de la columna 'Posted' para después transformarlo a formato fecha y que sea más fácil de generar las búsquedas. Elimino las filas que no tienen año.**

In [10]:
fecha = pd.to_datetime(reviews_clean.posted.str.replace('Posted', ''), format='mixed', errors='coerce')
fecha.head()

0   2011-11-05
1   2011-07-15
2   2011-04-21
3   2014-06-24
4   2013-09-08
Name: posted, dtype: datetime64[ns]

In [24]:
fecha.dt.year.value_counts()

posted
2014.0    22111
2015.0    18470
2013.0     6792
2012.0     1215
2011.0      532
2010.0       66
Name: count, dtype: int64

In [31]:
fecha.isnull().value_counts()

posted
False    49186
True     10147
Name: count, dtype: int64

In [11]:
reviews_clean['posted'] =fecha # Le asigno al df de reviews los valores limpios

**Elimino las filas donde no tenemos fecha (aproximadamente 10.000 filas). Ya que no nos van a servir para las funciones solicitadas**

In [12]:
reviews_clean.shape

(59333, 9)

In [13]:
reviews_clean.dropna(subset=['posted'], inplace=True)
reviews_clean.shape

(49186, 9)

In [62]:
reviews_clean.drop(columns= ['user_url', 'last_edited'], inplace=True)

In [63]:
reviews_clean.head()

,user_id,funny,posted,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [70]:
#exporto el df final como un csv
reviews_clean.to_csv('reviews_final.csv', index=False)

In [14]:
data = []
with gzip.open('users_items.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))
f.close()
items = pd.DataFrame(data)

In [54]:
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [55]:
items['items'].iloc[2]

[{'item_id': '1200',
  'item_name': 'Red Orchestra: Ostfront 41-45',
  'playtime_forever': 923,
  'playtime_2weeks': 0},
 {'item_id': '1230',
  'item_name': 'Mare Nostrum',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '1280',
  'item_name': "Darkest Hour: Europe '44-'45",
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '1520',
  'item_name': 'DEFCON',
  'playtime_forever': 158,
  'playtime_2weeks': 0},
 {'item_id': '220',
  'item_name': 'Half-Life 2',
  'playtime_forever': 1323,
  'playtime_2weeks': 0},
 {'item_id': '320',
  'item_name': 'Half-Life 2: Deathmatch',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '340',
  'item_name': 'Half-Life 2: Lost Coast',
  'playtime_forever': 90,
  'playtime_2weeks': 0},
 {'item_id': '360',
  'item_name': 'Half-Life Deathmatch: Source',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '380',
  'item_name': 'Half-Life 2: Episode One',
  'playtime_forever': 234,
  'playtime_2weeks': 0},
 

In [56]:
items.items_count.mean() #promedio de filas dentro la columna items

58.353629260559394

In [80]:
items.shape

(88310, 5)

In [57]:
items.items_count.mean() * items.shape[0] #como quedaría el dataframe si desanido esas filas

5153209.0

**Me voy a quedar solo con los 1000 jugadores que más juegos jugaron y que hicieron reviews**

In [15]:
items_filtrados = items[items['user_id'].isin(reviews_clean.user_id.unique())]

In [80]:
items_filtrados.shape

(23074, 5)

In [16]:
items_final = items_filtrados.nlargest(1000, 'items_count')

**Desanido la columna items. La lógica es que la que ya usamos para desanidar la columna reviews**

In [31]:
items_final['items'].iloc[0]

{'item_id': '300',
 'item_name': 'Day of Defeat: Source',
 'playtime_forever': 78,
 'playtime_2weeks': 0}

In [42]:
items_final = items_final.explode('items') # armo una fila por cada elemento dentro de la lista reviews
items_final.index = range(len(items_final)) #reseteo el index
items__desanidados = pd.json_normalize(items_final['items']) # armo un DF, con cada clave del diccionario dentro de items como una columna
items__desanidados.drop(columns='playtime_2weeks', inplace=True) # elimino la columna playtime_2weeks que no me sirve para las funciones
items_final = pd.concat([items_final.drop(columns= ['items', 'steam_id', 'items_count', 'user_url']), items__desanidados], axis=1) #concateno el df extendido  con el df con una columna por key de item

**Me quedo solo con las filas donde las horas de juego no son 0**

In [54]:
items_final_pulido =items_final[items_final['playtime_forever']!=0]

In [55]:
items_final_pulido.head()

,user_id,item_id,item_name,playtime_forever
0,phrostb,300,Day of Defeat: Source,78
13,phrostb,1900,Earth 2160,155
29,phrostb,3810,BloodRayne,404
30,phrostb,3820,BloodRayne 2,115
31,phrostb,3800,Advent Rising,165


In [56]:
items_final_pulido.to_csv('items_final_pulido.csv', index=False) #guardamos el df final en un csv para usar.